## Setup
### Project setup

In [91]:
if run_init:
    %pip install -U pip
    !if  [ ! -d "deep-learning-project" ] ; then git clone https://github.com/albertsgarde/deep-learning-project.git; fi
    !cd deep-learning-project && git reset --hard && git pull
    !source deep-learning-project/setup.sh deep-learning-project
run_init = False

In [92]:
run_init = True

In [93]:
run_init = False

### Imports

In [94]:
import numpy as np
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as nn_func
import torch.optim as optim
from torch.autograd import Variable

import audio_samples_py as aus

### Device setup

In [95]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Running GPU.") if use_cuda else print("No GPU available.")

Running GPU.


## Data
### Parameters

In [96]:
SAMPLE_LENGTH = 256

### Generation

In [97]:
import warnings

class AudioDataSet(torch.utils.data.Dataset):
    def __init__(self, parameters: aus.DataParameters):
         self.parameters = parameters

    def __len__(self):
        return np.iinfo(np.int64).max
    
    def __getitem__(self, index):
        data_point = self.parameters.generate_at_index(index)
        return data_point.get_samples(), torch.tensor([data_point.get_frequency_map()])



In [98]:

data_loader_params = {"batch_size": 1}

parameters = aus.DataParameters(num_samples=SAMPLE_LENGTH).add_sine((0.5,0.75))
training_parameters = parameters.with_seed_offset(0)
training_generator = aus.DataGenerator(training_parameters)
training_loader = torch.utils.data.DataLoader(AudioDataSet(training_parameters), **data_loader_params)
validation_parameters = parameters.with_seed_offset(1)
validation_generator = aus.DataGenerator(validation_parameters)
validation_loader = torch.utils.data.DataLoader(AudioDataSet(validation_parameters), **data_loader_params)




## Neural Network

In [99]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5, stride=1, padding=2)
        self.lin_out = nn.Linear(in_features=SAMPLE_LENGTH, out_features=1)
        
        
    def forward(self, x):
        x = x.unsqueeze(0)
        x = self.conv(x)
        x = x.flatten()
        x = nn_func.relu(x)
    
        return self.lin_out(x).unsqueeze(1)

net = Net()
if use_cuda:
    net.cuda()

## Training

In [100]:
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5

In [101]:
criterion = nn.MSELoss()  

# weight_decay is equal to L2 regularization
optimizer = optim.AdamW(net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY) 


In [102]:
def to_torch(x):
    variable = Variable(torch.from_numpy(x))
    if use_cuda:
        variable = variable.cuda()
    return variable

def test_net(net, validation_loader, criterion):
    was_training = net.training
    net.eval()
    num_iterations = 100
    total_loss = 0
    for data_point, frequency in itertools.islice(validation_loader, num_iterations):
        data_point = data_point.to(device)
        frequency = frequency.to(device)
        output = net(data_point)
        total_loss += criterion(output, frequency)
    net.train(mode=was_training)
    return total_loss/num_iterations

def manual_test(net, validation_generator):
    was_training = net.training
    net.eval()
    num_iterations = 5
    for data_point in validation_generator.next_n(num_iterations):
        samples = to_torch(data_point.get_samples())
        freq_map = to_torch(np.array([data_point.get_frequency_map()]))
        output = net(samples)
        print("Frequency: ", parameters.map_to_frequency(freq_map.item()), " Output: ", parameters.map_to_frequency(output.item()))
    net.train(mode=was_training)



In [103]:
NUM_BATCHES = 10000
EVAL_EVERY = 1000


manual_test(net, validation_generator)

net.train()

for i, (data_point, frequency) in enumerate(itertools.islice(training_loader, NUM_BATCHES)):
    if i%EVAL_EVERY == 0:
        print(f"Loss at iteration {i}: {test_net(net, validation_loader, criterion)}")
    data_point = data_point.to(device)
    frequency = frequency.to(device)

    output = net(data_point)
    loss = criterion(output, frequency)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f"Loss at iteration {NUM_BATCHES}: {test_net(net, validation_loader, criterion)}")
manual_test(net, validation_generator)
    

Frequency:  21.806589126586914  Output:  666.97705078125
Frequency:  7476.90869140625  Output:  666.97705078125
Frequency:  50.53485107421875  Output:  666.97705078125
Frequency:  10745.4326171875  Output:  666.97705078125
Frequency:  18996.27734375  Output:  776.4674072265625
Loss at iteration 0: 0.34472009539604187
Loss at iteration 1000: 0.09355480223894119
Loss at iteration 2000: 0.04979756101965904
Loss at iteration 3000: 0.0254303477704525
Loss at iteration 4000: 0.02380792237818241
Loss at iteration 5000: 0.025056447833776474
Loss at iteration 6000: 0.020336538553237915
Loss at iteration 7000: 0.021875951439142227
Loss at iteration 8000: 0.01827203668653965
Loss at iteration 9000: 0.023712018504738808
Loss at iteration 10000: 0.01931377500295639
Frequency:  260.94561767578125  Output:  273.6207275390625
Frequency:  15659.8359375  Output:  9962.0595703125
Frequency:  5092.42333984375  Output:  16707.51171875
Frequency:  193.81607055664062  Output:  225.78265380859375
Frequency:  